In [ ]:
!pip install flask --break-system-packages
!pip install flask-restx  --break-system-packages

## **Example API System**

### **RESTful API Features**

#### **1. General Greeting**
- **GET `/example/hello`**
  - **Description**: Returns a generic greeting message.
  - **Response Schema/Model**:
    - `message`: string 
  - **HTTP Status Codes**:
    - `200 OK`: Successful response.

#### **2. Personalized Greeting**
- **GET `/example/greet/<string:name>`**
  - **Description**: Returns a personalized greeting message using the `name` path parameter.
  - **Path Parameters**:
    - `name` (string): The name of the person to greet.
  - **Response Schema/Model**: 
    - `name`: string,
    - `message`: string 
  - **HTTP Status Codes**:
    - `200 OK`: Successful response.

#### **3. Query-Based Greeting**
- **GET `/example/query`**
  - **Description**: Returns a greeting message based on query parameters (`name` and `age`).
  - **Query Parameters**:
    - `name` (string, optional): Name of the person. Defaults to `"Guest"`.
    - `age` (integer, optional): Age of the person.
  - **Response Schema/Model**:  
    - `name` (string, optional): Name of the person. Defaults to `"Guest"`.
    - `age` (integer, optional): Age of the person.
    - `message`: str, 
  - **HTTP Status Codes**:
    - `200 OK`: Successful response.
    - `400 Bad Request`: Invalid query parameters.
---

 


In [ ]:
import nest_asyncio
from flask import Flask, request
from flask_restx import Api, Resource, fields
from werkzeug.serving import run_simple

nest_asyncio.apply()

# initialize Flask app and FLASK RESTX API
app = Flask(__name__)
api = Api(app, version="1.0", title="Flask RESTX API", description="A demonstration of Flask-RESTX with class-based routes")

example_ns = api.namespace('example', description='Endpoints demonstrating basic Flask-RESTX functionality')

# model has two uses 
# 1. for marshaling to make the response network transportable
# 2. for validating checks if the response matches the model

# hello endpoint response model 
hello_model = example_ns.model('hello', {
    "message": fields.String(description="Greeting message", example="Hello, World!")
})

# greet endpoint response model
greet_model = example_ns.model('Greet', {
    'name': fields.String(required=True, description="Name of the person being greeted", example="Adam"),
    'message': fields.String(description="Personalized greeting message", example="Hello, Adam!")
})

# query endpoint response model
query_model = example_ns.model('Query', {
    'name': fields.String(description="Name parameter from query string", example="Adam"),
    'age': fields.Integer(description="Age parameter from query string", example=30),
    'message': fields.String(description="Response message", example="Hello, Adam! You are 30 years old.")
})

@example_ns.route('/hello')
class HelloWorld(Resource):
  
    @example_ns.marshal_with(hello_model)
    def get(self):
        """Returns a generic greeting message."""
        
        return {"message": " Hello, world"}, 200
        
 
@example_ns.route('/greet/<string:name>')
@example_ns.param('name', description='Name of the person to greet')
class Greet(Resource):
    
    @example_ns.marshal_with(greet_model)
    def get(self, name: str):
        """Returns a personalized greeting message using the `name` path parameter."""
        
        return {"name": name, "message": f"Hello {name}!"}

# http://localhost:5000/example/query?name="adam"&age=30
@example_ns.route('/query')
class Query(Resource):
    @example_ns.doc(params={
     'name': {'description': 'Name parameter (optional)', 'default': 'Guest'},
     'age': {'description': 'Age parameter (optional)', 'type': 'integer'}
    })
    @example_ns.marshal_with(query_model)
    def get(self):
        """Returns a greeting message based on query parameters (`name` and `age`)."""
        name = request.args.get("name", "Guest")
        age = request.args.get("age")
        
        
        # Check if 'age' is provided and construct the message accordingly
        if age:
            return {"name": name, "age": int(age), "message": f"Hello, {name}! You are {age} years old."}, 200

        return {"name": name, "message": f"Hello, {name}!"}, 200



api.add_namespace(example_ns)

# Run  Flask in Jupyter Notebook 
run_simple('localhost', 5000, app)


## **Comments API System**

### **RESTful API Features**

#### **1. Create Comment**
- **POST `/comment`**
  - **Description**: Creates a new comment linked to a post.
  - **Request Schema/Model**:
    - `id`: integer (required) - Unique ID of the comment.
    - `content`: string (required) - Content of the comment.
    - `post_id`: integer (required) - ID of the associated post.
  - **Response Schema/Model**:
    - `success`: string
    - `data`: object
      - `id`: integer
      - `content`: string
      - `post_id`: integer
  - **HTTP Status Codes**:
    - `201 Created`: Comment successfully created.
    - `400 Bad Request`: Invalid input.
    - `409 Conflict`: Comment with this ID already exists.

#### **2. Get Comment**
- **GET `/comment/<int:comment_id>`**
  - **Description**: Retrieves a comment by its ID.
  - **Path Parameters**:
    - `comment_id`: integer (required) - Unique ID of the comment to retrieve.
  - **Response Schema/Model**:
    - `id`: integer
    - `comment`: object
      - `id`: integer
      - `content`: string
      - `post_id`: integer
  - **HTTP Status Codes**:
    - `200 OK`: Comment successfully retrieved.
    - `404 Not Found`: Comment not found.

#### **3. Update Comment**
- **PUT `/comment/<int:comment_id>`**
  - **Description**: Updates the content and associated post ID of an existing comment.
  - **Path Parameters**:
    - `comment_id`: integer (required) - Unique ID of the comment to update.
  - **Request Schema/Model**:
    - `content`: string (required) - Updated content of the comment.
    - `post_id`: integer (required) - Updated ID of the associated post.
  - **Response Schema/Model**:
    - `id`: integer
    - `comment`: object
      - `content`: string
      - `post_id`: integer
  - **HTTP Status Codes**:
    - `200 OK`: Comment successfully updated.
    - `400 Bad Request`: Invalid input.
    - `404 Not Found`: Comment not found.

#### **4. Delete Comment**
- **DELETE `/comment/<int:comment_id>`**
  - **Description**: Deletes a comment by its ID.
  - **Path Parameters**:
    - `comment_id`: integer (required) - Unique ID of the comment to delete.
  - **Response Schema/Model**:
    - `success`: string
  - **HTTP Status Codes**:
    - `204 No Content`: Comment successfully deleted.
    - `404 Not Found`: Comment not found.

---

### **Models**
#### **Comment Data**
- `id` (int): Unique identifier for the comment.
- `content` (str): Content of the comment.
- `post_id` (int): ID of the associated post.

---



In [14]:
import nest_asyncio
from werkzeug.serving import run_simple
from flask import Flask
from flask_restx import Api, Namespace, Resource, fields
from http import HTTPStatus

nest_asyncio.apply()

# Initialize Flask app and API
app = Flask(__name__)

# Wrap the initialized app with Flask-Restx swagger
api  = Api(app, version="1.0", title="Comments API System", description="A Flask-RESTX API for managing comments.")

# In-memory storage
comments = {}
""" 
 {
     "?": { }
 }
"""

# Initialize name space for grouping endpoints
comments_ns = api.namespace('comments', description="Operations related to comments")
 
# commentUpdateRequest Model 
comment_update_request_model = api.model('commentUpdateRequest',{                            
        'content': fields.String(description="Content of the comment"),
        'post_id': fields.Integer(description="ID of the associated post")
})

# commentData Model
comment_data_model =  api.model('CommentData',
    {             
        'id': fields.Integer(required=True, description="Unique ID of the comment"),
        'content': fields.String(required=True, description="Content of the comment"),
        'post_id': fields.Integer(required=True, description="ID of the associated post")
    }
)

# CommentResponse Model
comment_response_model =  comments_ns.model('CommentResponse', {
    'success': fields.String(description="Status of the operation"),
    "data": fields.Nested(comment_data_model)
   
})

# Define Resources
@comments_ns.route("/") # post
class CommentsCollection(Resource):
    
    @comments_ns.expect(comment_data_model, validate=True)
    @comments_ns.response(HTTPStatus.CREATED, "Comment successfully created", comment_response_model)
    @comments_ns.response(HTTPStatus.BAD_REQUEST, "Invalid input")
    @comments_ns.response(HTTPStatus.CONFLICT, "Comment with this ID already exists" )
    def post(self):
        """Creates a new comment linked to a post"""
        data = api.payload
        comment_id = data["id"] 
        
        # if "id" not in data or "content" not in data or "post_id" not in data:
        #     response_data = {"error", "Invalid input"}
        #     response_status_code = HTTPStatus.BAD_REQUEST
            
        #     return response_data, response_status_code
        #     return {"error", "Invalid input"}, HTTPStatus.BAD_REQUEST
            
        if comment_id in comments:
            response_data = {"error", "Comment with this ID already exists"}
            response_status_code = HTTPStatus.CONFLICT
            
            return response_data, response_status_code
            # return {"error", "Comment with this ID already exists"}, HTTPStatus.CONFLICT     
        
        comments[comment_id] = data  
    
        response_data = {"success": "Comment created", "data": data},
        response_status_code = HTTPStatus.CREATED
            
        return response_data, response_status_code
        # return {"success": "Comment created", "data": data}, HTTPStatus.CREATED
            
        

@comments_ns.route("/<int:comment_id>")
class CommentResource(Resource):
    
    @comments_ns.response(HTTPStatus.OK, "Comment successfully retrieved", comment_data_model)
    @comments_ns.response(HTTPStatus.NOT_FOUND, "Comment not found")
    def get(self, comment_id: int):
        """Retrieves a comment by its ID"""
        if comment_id not in comments:
            response_data = {"error": "Comment not found"}
            response_status_code = HTTPStatus.NOT_FOUND
            
            return response_data, response_status_code
            # return {"error": "Comment not found"}, HTTPStatus.NOT_FOUND

        response_data = comments[comment_id]
        response_status_code = HTTPStatus.OK
        
        return response_data, response_status_code
        # return {"id": comment_id, "comment": comments[comment_id]}, HTTPStatus.OK


            
    @comments_ns.expect(comment_update_request_model, validate=True)
    @comments_ns.response(HTTPStatus.OK, "Comment successfully updated", comment_data_model)
    @comments_ns.response(HTTPStatus.BAD_REQUEST, "Invalid input")
    @comments_ns.response(HTTPStatus.NOT_FOUND, "Comment not found")
    def put(self, comment_id:int):
        """Update an existing comment"""
        if comment_id not in comments:
            response_data = {"error": "Comment not found"}
            response_status_code = HTTPStatus.NOT_FOUND
            
            return response_data, response_status_code
            # return {"error": "Comment not found"}, HTTPStatus.NOT_FOUND
            
        
        """ 
        comment_id=1
        data = {
            id: 1, 
            content: "123123112312312312131"
        }
        comments[comment_id]  = {
            id:1
            content: "12312312"
            post_id: 11
        }
        **comments[comment_id] = id:1, content: "12312312", ost_id: 11
        **data = id: 1, content: "123123112312312312131"
        comments[comment_id]  = {**comments[comment_id], **data } = {id:1, content: "12312312", ost_id: 11, id: 1, content: "123123112312312312131"}
        now clear with override
        comments[comment_id]  = {**comments[comment_id], **data } = { ost_id: 11, id: 1, content: "123123112312312312131"}
    
        """
        
        data = api.payload
        comments[comment_id] = {**comments[comment_id], **data }
    
        response_data =  data
        response_status_code = HTTPStatus.OK
        
        return response_data, response_status_code

    @comments_ns.response(HTTPStatus.NO_CONTENT, "Comment successfully deleted")
    @comments_ns.response(HTTPStatus.NOT_FOUND, "Comment not found")
    def delete(self, comment_id:int):
        """Delete a comment by ID"""
        if comment_id not in comments:
            response_data = {"error": "Comment not found"}
            response_status_code = HTTPStatus.NOT_FOUND
            
            return response_data, response_status_code
            # return {"error": "Comment not found"}, HTTPStatus.NOT_FOUND
        
        del comments[comment_id]
        
        response_data =  {"success": "Comment deleted"}
        response_status_code = HTTPStatus.NO_CONTENT
        
        return response_data, response_status_code
        # return {"success": "Comment deleted"}, HTTPStatus.NO_CONTENT
        
    
# add name space to API
api.add_namespace(comments_ns, path="/comment")

run_simple("localhost", 5000, app)

 * Running on http://localhost:5000
Press CTRL+C to quit
127.0.0.1 - - [08/Jan/2025 21:03:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Jan/2025 21:03:02] "GET /swaggerui/droid-sans.css HTTP/1.1" 304 -
127.0.0.1 - - [08/Jan/2025 21:03:02] "GET /swaggerui/swagger-ui.css HTTP/1.1" 304 -
127.0.0.1 - - [08/Jan/2025 21:03:02] "GET /swaggerui/swagger-ui-bundle.js HTTP/1.1" 304 -
127.0.0.1 - - [08/Jan/2025 21:03:02] "GET /swaggerui/swagger-ui-standalone-preset.js HTTP/1.1" 304 -
127.0.0.1 - - [08/Jan/2025 21:03:03] "GET /swagger.json HTTP/1.1" 200 -
127.0.0.1 - - [08/Jan/2025 21:03:03] "GET /swaggerui/favicon-32x32.png HTTP/1.1" 304 -
127.0.0.1 - - [08/Jan/2025 21:03:13] "POST /comments/ HTTP/1.1" 201 -
127.0.0.1 - - [08/Jan/2025 21:03:24] "DELETE /comments/1 HTTP/1.1" 204 -
